#### Import package yang diperlukan

In [126]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import sleep

#### Beri variabel nama website dan uji koneksi website dan hasil status codenya

In [120]:
website = 'https://www.etsy.com/c/accessories?ref=catnav-1'

In [121]:
response = requests.get(website)
response.status_code

200

#### Ubah object bs4 dan beri nama variabel soup untuk melakukan parse html

In [39]:
soup = BeautifulSoup(response.content, 'html.parser')
soup

<!DOCTYPE html>

<html lang="en-US" xmlns:fb="https://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<script>if (window.performance && performance.mark) performance.mark("TTP")</script>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="en-US" http-equiv="content-language"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="nosearch" name="pinterest"/>
<meta content="3:1705559399:emJbln6jjmr7gXOlBLmHA1Gj4n6E:2916186a6059636a9abd286c2f868ed88bbb301ea945c561d972a0f233f6f2b8" name="csrf_nonce"/>
<meta content="3:1705559399:9rM12wiub692bq6ud8EpGgOUTamG:ca045a1bcc389a39e7554a30829631ece2bb9ff20cb51c5c4423cc036f025798" name="uaid_nonce"/>
<meta content="89186614300" property="fb:app_id"/>
<meta content="/ac/sasquatch/css/" name="css_dist_path">
<meta content="202401171705534848" name="dist">
<script nonce="/L9T66319FNlTNoBwjDlzZsP">
    !function(){var e=window.__etsy_logging={};e.errorQueue=[],

#### Mencari elemen `<li class="wt-list-unstyled .....">`

In [40]:
# Find all elements with class starting with 'wt-order-xs-'
results = soup.find_all('li', class_=lambda x: x and x.startswith('wt-list-unstyled'))

#### Melihat jumlah item pada halaman tersebut

In [41]:
len(results)

48

#### Cari tiap elemen :  
Pada praktik kali ini saya akan mengambil data :  
- nama produk
- rating
- jumlah review
- harga
- mata uang
- nama penjual
- link produk

#### Di bawah ini kita coba dahulu untuk data pertama/array[0]

In [42]:
#Product Name
productname = results[0].find('h3', class_=lambda x: x and x.startswith('wt-text-caption')).get_text().strip()

In [47]:
#Star
productstar = results[0].find('div', {'class':'sprite-img'}).get('aria-label')

In [46]:
#Reviews
reviewcount = results[0].find('span', {'class':'wt-text-caption'}).get_text().strip()

In [45]:
#Price
priceamount = results[0].find('span', {'class':'currency-value'}).get_text()

In [44]:
#Price Symbol
pricecurrency = results[0].find('span', {'class':'currency-symbol'}).get_text()

In [138]:
#Seller
# Find the p element
p_element = results[0].find('p', class_=lambda x: x and x.startswith('wt-text-caption wt-text-truncate'))
# Find all span elements within the p element
span_elements = p_element.find_all('span')
#Get text from span class second from the last
span_text = span_elements[-2].text

In [139]:
def extract_text_from_results(results):
    # Find the p element
    p_element = results.find('p', class_=lambda x: x and x.startswith('wt-text-caption wt-text-truncate'))

    # Check if the p element is found
    if p_element:
        # Find all span elements within the p element
        span_elements = p_element.find_all('span')

        # Check if there are at least two span elements
        if len(span_elements) >= 2:
            # Get text from the second-to-last span element
            span_text = span_elements[-2].text
            return span_text
        else:
            return "Not enough span elements within the p element."
    else:
        return "P element not found."

# Example usage:
result = result # Replace with the actual result
text_from_span = extract_text_from_results(result)

In [14]:
#Link
productlink = results[0].find('a', {'class':'listing-link'}).get('href')

#### Setelah tidak ada error yang muncul, maka sekarang kita ambil untuk semua 48 produk tersebut.

In [95]:
product_name = []
star_rating = []
review_count = []
product_price = []
currency = []
seller_name = []
product_link = []

for result in results:
    try:
        product_name.append(result.find('h3', class_=lambda x: x and x.startswith('wt-text-caption')).get_text().strip())
    except:
        product_name.append('n/a')

for result in results:
    try:
        star_rating.append(result.find('div', {'class':'sprite-img'}).get('aria-label'))
    except:
        star_rating.append('n/a')

for result in results:
    try:
        review_count.append(result.find('span', {'class':'wt-text-caption'}).get_text().strip())
    except:
        review_count.append('n/a')

for result in results:
    try:
        product_price.append(result.find('span', {'class':'currency-value'}).get_text())
    except:
        product_price.append('n/a')

for result in results:
    try:
        currency.append(result.find('span', {'class':'currency-symbol'}).get_text())
    except:
        currency.append('n/a')

for result in results:
    try:
        text_from_span = extract_text_from_results(result)
        seller_name.append(text_from_span)
    except Exception as e:
        print(f"Error processing result: {e}")
        seller_name.append('n/a')

for result in results:
    try:
        product_link.append(result.find('a', {'class':'listing-link'}).get('href'))
    except:
        product_link.append('n/a')

#### Setelah 48 data produk diambil, maka sekarang kita ubah menjadi DataFrame

In [96]:
product_overview = pd.DataFrame({'name':product_name, 'rating':star_rating,'review':review_count,'price':product_price,
                                'currency':currency,'seller':seller_name,'link':product_link})
product_overview

,name,rating,review,price,currency,seller,link
0,"Minimalist gold hair accessories, brass hair c...",5 out of 5 stars,"(8,141)",15.30,$,annielesperance,https://www.etsy.com/listing/463426102/minimal...
1,"Infinity Scarf, Cowl, Wool Scarf, Snood -- LE ...",5 out of 5 stars,(478),75.00,$,deroucheau,https://www.etsy.com/listing/160217560/infinit...
2,Custom hand punched brass tag - hand stamped ...,5 out of 5 stars,"(21,675)",8.00,$,aVintageParcel,https://www.etsy.com/listing/245546042/custom-...
3,"Personalized Money Clip, Full Name, Rustic, Cu...",5 out of 5 stars,"(4,679)",18.00,$,321SimpleCreations,https://www.etsy.com/listing/506599156/persona...
4,"Iron anniversary gift, 6th anniversary key cha...",5 out of 5 stars,"(2,854)",26.00,$,AdirondackIron,https://www.etsy.com/listing/507901564/iron-an...
5,Choose your State or Country Custom Long Dista...,5 out of 5 stars,"(10,210)",26.35,$,thelightandthedark1,https://www.etsy.com/listing/193512457/choose-...
6,"Custom Leather Keychain, Personalized Leather ...",5 out of 5 stars,"(5,657)",12.50,$,madebyperri,https://www.etsy.com/listing/232958676/custom-...
7,"Personalized leather keychain, Custom Keychain...",5 out of 5 stars,"(18,164)",12.32,$,IRONandGRAINusa,https://www.etsy.com/listing/493595319/persona...
8,Yeti Bigfoot Sasquatch Rustic Steel Metal Key ...,5 out of 5 stars,"(22,084)",12.50,$,rvmetalshop,https://www.etsy.com/listing/523288167/yeti-bi...
9,Monogram Hat Womens Personalized Baseball Cap ...,5 out of 5 stars,"(105,471)",16.99,$,EverlyGrayce,https://www.etsy.com/listing/196232414/monogra...


#### Setelah sudah menjadi bentuk dataframe, maka selanjutnya kita export menjadi tipe Excel.

In [98]:
product_overview.to_excel('product_scraping.xlsx', index=False)

#### Dibawah ini adalah scraping secara Multiple Pages

In [129]:
product_name = []
star_rating = []
review_count = []
product_price = []
currency = []
seller_name = []
product_link = []
web = []
resultcount = []

max_retries = 5
for i in range (1,21):
    website = 'https://www.etsy.com/c/accessories?ref=pagination&page=' + str(i)
    retries = 0
    while retries < max_retries:
        response = requests.get(website)
            
        if response.status_code == 200:
            # The request was successful, break out of the retry loop
            break
        else:
            # Wait for a moment before retrying
            sleep(5)
            retries += 1
            print(f"Retrying {website}, attempt {retries}")
            
    soup = BeautifulSoup(response.content, 'html.parser')
    results = soup.find_all('li', class_=lambda x: x and x.startswith('wt-list-unstyled'))
    resultcount.append(len(results))
    for result in results:
        try:
            product_name.append(result.find('h3', class_=lambda x: x and x.startswith('wt-text-caption')).get_text().strip())
        except:
            product_name.append('n/a')
    
        try:
            star_rating.append(result.find('div', {'class':'sprite-img'}).get('aria-label'))
        except:
            star_rating.append('n/a')
    
        try:
            review_count.append(result.find('span', {'class':'wt-text-caption'}).get_text().strip())
        except:
            review_count.append('n/a')
    
        try:
            product_price.append(result.find('span', {'class':'currency-value'}).get_text())
        except:
            product_price.append('n/a')
    
        try:
            currency.append(result.find('span', {'class':'currency-symbol'}).get_text())
        except:
            currency.append('n/a')
    
        try:
            text_from_span = extract_text_from_results(result)
            seller_name.append(text_from_span)
        except Exception as e:
            print(f"Error processing result: {e}")
            seller_name.append('n/a')
    
        try:
            product_link.append(result.find('a', {'class':'listing-link'}).get('href'))
        except:
            product_link.append('n/a')

Retrying https://www.etsy.com/c/accessories?ref=pagination&page=1, attempt 1
Retrying https://www.etsy.com/c/accessories?ref=pagination&page=1, attempt 2
Retrying https://www.etsy.com/c/accessories?ref=pagination&page=9, attempt 1
Retrying https://www.etsy.com/c/accessories?ref=pagination&page=9, attempt 2
Retrying https://www.etsy.com/c/accessories?ref=pagination&page=9, attempt 3
Retrying https://www.etsy.com/c/accessories?ref=pagination&page=11, attempt 1
Retrying https://www.etsy.com/c/accessories?ref=pagination&page=12, attempt 1
Retrying https://www.etsy.com/c/accessories?ref=pagination&page=12, attempt 2
Retrying https://www.etsy.com/c/accessories?ref=pagination&page=12, attempt 3
Retrying https://www.etsy.com/c/accessories?ref=pagination&page=13, attempt 1
Retrying https://www.etsy.com/c/accessories?ref=pagination&page=13, attempt 2
Retrying https://www.etsy.com/c/accessories?ref=pagination&page=13, attempt 3
Retrying https://www.etsy.com/c/accessories?ref=pagination&page=14, a

In [131]:
product_overview = pd.DataFrame({'name':product_name, 'rating':star_rating,'review':review_count,'price':product_price,
                                'currency':currency,'seller':seller_name,'link':product_link})
product_overview

,name,rating,review,price,currency,seller,link
0,"Minimalist gold hair accessories, brass hair c...",5 out of 5 stars,"(8,141)",15.30,$,annielesperance,https://www.etsy.com/listing/463426102/minimal...
1,"Infinity Scarf, Cowl, Wool Scarf, Snood -- LE ...",5 out of 5 stars,(478),75.00,$,deroucheau,https://www.etsy.com/listing/160217560/infinit...
2,"Personalized Money Clip, Full Name, Rustic, Cu...",5 out of 5 stars,"(4,679)",18.00,$,321SimpleCreations,https://www.etsy.com/listing/506599156/persona...
3,"Custom Leather Keychain, Personalized Leather ...",5 out of 5 stars,"(5,657)",12.50,$,madebyperri,https://www.etsy.com/listing/232958676/custom-...
4,"Iron anniversary gift, 6th anniversary key cha...",5 out of 5 stars,"(2,854)",26.00,$,AdirondackIron,https://www.etsy.com/listing/507901564/iron-an...
...,...,...,...,...,...,...,...
811,KNIGHT Armor Mask - Make your own Knight's hel...,5 out of 5 stars,(898),6.28,$,Ntanos,https://www.etsy.com/listing/512829402/knight-...
812,"Beagle Dog 3D Papercraft Mask Template, Low Po...",5 out of 5 stars,"(4,788)",8.58,$,Wintercroft,https://www.etsy.com/listing/199870596/beagle-...
813,Woodland leaf tiara hairband for greek goddess...,5 out of 5 stars,(961),72.56,$,SeegangBerlin,https://www.etsy.com/listing/508003395/woodlan...
814,"Infant Hair Clips, Baby Hair Clips for Fine Ha...",5 out of 5 stars,"(1,112)",8.31,$,BambiLaneAccessories,https://www.etsy.com/listing/464631201/infant-...


#### Export to Excel

In [132]:
product_overview.to_excel('multipage_etsy_accessories.xlsx', index=False)